In [2]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent
from sympy import symbols, bool_map
from sympy.logic.inference import satisfiable
from itertools import combinations
from belief_agent import BeliefBase
from math import factorial

**KB: Robert does well in the exam if and only if he is prepared or lucky.**
Robert does not do well the exam.

**Question: KB |= φ?**  **(KB entails phi)**

1. (-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ $\neg$ r, formalisation of KB as CNF.

2.(-r $\vee$ p $\vee$ s) $\wedge$ (p $\wedge$ r) $\wedge$ (s $\wedge$ r) $\wedge$ r $\wedge$ p, CNF for contradiction.

3. clauses:= (-r $\vee$ p $\vee$ s), (p $\wedge$ r), (s $\wedge$ r), r, p.

4. clauses:=clauses U {r} from resolving (¬p $\vee$ r) with p.

5. clauses:=clauses U {1} from resolving r with $\neg$ r.

6. Result: empty clause, so KB |= φ.

In [3]:
belief_base = BeliefBase('q p')
symb = belief_base.symbols
q, p = symb

belief_base.expand(p, 1)
belief_base.expand(q, 2)
belief_base.expand(Implies(p, q), 3)
print("Belief Base:", belief_base)

Belief Base: <belief_agent.BeliefBase object at 0x112ec38b0>


In [32]:
from sympy.logic.boolalg import And, Or, Not, Implies, Equivalent, to_cnf
from sympy import symbols
from sympy.logic.inference import satisfiable
from utils import generate_combinations

class BeliefBase:
    def __init__(self, atomic_symbols: str):
        self.beliefs = set()
        self.beliefs_priorities = set()
        self.symbols = symbols(atomic_symbols)
        self.priorities = {}

    def expand(self, belief, priority):
        cnf_belief = to_cnf(belief)
        self.beliefs.add((cnf_belief))
        self.beliefs_priorities.add((cnf_belief, priority))
        self.priorities[belief] = priority

    def remove(self, belief, belief_set=None):
        cnf_belief = to_cnf(belief)
        if belief_set is None:
            if cnf_belief in self.beliefs:
                self.beliefs.remove(cnf_belief)
        else:
            if cnf_belief in belief_set:
                belief_set.remove(cnf_belief)

    def contract(self, extract_belief):
        sorted_beliefs = sorted(self.beliefs_priorities, key=lambda x: (-x[1]))
        while self.check_entailment(extract_belief):
            for (belief, pri) in sorted_beliefs:
                print("set: ", set([belief]))
                belief_is_entailed = self.check_entailment(formula=extract_belief, belief_base=belief)
                print("belief_is_entailed:", belief_is_entailed)
                if belief_is_entailed == True:
                    print("belief set before removal:", self.beliefs)
                    self.remove(belief)
                    sorted_beliefs.remove((to_cnf(belief), pri))
                    print("belief set after removal:", self.beliefs, "\n")
                    break

    def revise(self, belief):
        # Levi's identity
        self.contract(Not(belief))
        self.expand(belief)

    def check_entailment(self, formula, belief_base=None):
        if belief_base is None:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(self.beliefs)
            new_set = And(*belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)
        else:
            cnf_formula = to_cnf(formula)
            belief_base_cnf = to_cnf(belief_base)
            new_set = And(belief_base_cnf, Not(cnf_formula))
            return not satisfiable(new_set)         

# Test
if __name__ == "__main__":
    belief_base = BeliefBase('p q')  
    p, q = belief_base.symbols
    # Expand belief base
    belief_base.expand(p, 1)
    belief_base.expand(q, 2)
    belief_base.expand(Implies(p, q), 3)
    belief_base.expand(Or(p, q), 4)
    print(belief_base.beliefs)
    
    print("\n Contraction:")
    belief_base.contract(q)
    print("\nNew beliefbalse:")
    print(belief_base.beliefs) 

{q | ~p, p | q, q, p}

 Contraction:
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {q}
belief_is_entailed: True
belief set before removal: {q | ~p, p | q, q, p}
belief set after removal: {q | ~p, p | q, p} 

set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q | ~p}
belief_is_entailed: False
set:  {p}
belief_is_entailed: False
set:  {p | q}
belief_is_entailed: False
set:  {q 

KeyboardInterrupt: 

In [30]:
print(belief_base.beliefs_priorities)
print(belief_base.priorities)

{(q, 2), (q | ~p, 3), (p, 1)}
{p: 1, q: 2, Implies(p, q): 3}


In [15]:
from itertools import product
from math import factorial

def generate_combinations(n):
    return list(product([True, False], repeat=n))

def neighbours(beliefs: set):
    if len(beliefs) == 1:
        return None

    elif len(beliefs) > 1:
        neigh = []
        for clause in beliefs:
            new_set = beliefs.copy()
            new_set.remove(clause)
            neigh.append(new_set)
        return neigh

def create_graph(belief_set):
    graph = dict()
    total_nodes_in_max_depth = factorial(len(belief_set))
    counter = 0
    current_set = belief_set
    while counter <= total_nodes_in_max_depth:
        neigh = neighbours(current_set)
        if neigh is None:
            counter += 1
    
        else:
            graph[tuple(current_set)] = [tuple(n) for n in neigh]  # Convert sets to tuples
            counter += 1
            current_set = neigh[0]  # Explore all neighbors
    print(f"Out of loop after {counter} iterations")
    return graph





In [16]:
create_graph(belief_base.beliefs)

Out of loop after 7 iterations


{(q | ~p, p, q): [(p, q), (q | ~p, q), (q | ~p, p)], (p, q): [(q,), (p,)]}